## Airbnb Data

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from urllib3.exceptions import ProtocolError
import pprint


In [118]:
from pandas.io.json import json_normalize

In [171]:
import pandas as pd

In [250]:
from urllib3.exceptions import ProtocolError

In [139]:
api_base = 'https://www.airbnb.com/api/v2/explore_tabs?version=1.3.8&_format=for_explore_search_web&experiences_per_grid=20&items_per_grid=18&guidebooks_per_grid=20&auto_ib=true&fetch_filters=true&has_zero_guest_treatment=false&is_guided_search=true&is_new_cards_experiment=true&luxury_pre_launch=false&query_understanding_enabled=true&show_groupings=true&supports_for_you_v3=true&timezone_offset=-360&client_session_id=82674acc-b274-41dc-8b00-499d5c2fea44&metadata_only=false&is_standard_search=true&refinement_paths%5B%5D=%2Fhomes&selected_tab_id=home_tab&place_id=ChIJ5S-raZElv0cR8HcqSvxgJwQ&allow_override%5B%5D=&s_tag=kke9hwU1&screen_size=medium&query=Cologne%2C+Germany&_intents=p1'
api_key = '&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&currency=USD&locale=en'
api_section_offset = '&section_offset=4'

In [13]:
import requests
import json

In [251]:
retry_lvl = [100, 0.1]

## Key

In [24]:
from bs4 import BeautifulSoup

In [81]:
airbnb_home = requests.get("http://airbnb.com")

soup = BeautifulSoup(airbnb_home.content, "html.parser")

metatags = soup.find_all('meta', id="_bootstrap-layout-init")
metacontents = [metatag["content"] for metatag in metatags]

metajson = json.loads(metacontents[0])

metajson

api_key = metajson['api_config']['key']
api_key = '&key='+api_key

In [82]:
print(api_key)

&key=d306zoyjsyarp7ifhu67rjxn52tv0t20


## Explore API

In [84]:
rsp = requests.get(api_base+api_key)

In [85]:
rsp_json = rsp.json()

In [87]:
len(rsp_json)

2

In [89]:
rsp_json.keys()

dict_keys(['explore_tabs', 'metadata'])

In [ ]:
"pagination_metadata": {
        "has_next_page": true,
        "items_offset": 18,
        "section_offset": 4,
        "search_session_id": "eee6b55b-100d-4314-800a-393bec147f25"
      }

In [138]:
rsp_json['explore_tabs'][0]['pagination_metadata']['has_next_page']

True

In [103]:
rsp_json['explore_tabs'][0]['home_tab_metadata'].keys()

dict_keys(['urgency_commitment', 'golden_ticket_urgency_commitment', 'listing_cards_urgency_commitment_metadata', 'listing_cards_price_line_urgency_commitment_metadata', 'messages', 'facets', 'overrides', 'search_feed_items', 'remarketing_ids', 'location', 'breadcrumbs', 'listings_count', 'search', 'geography', 'price_histogram', 'filters'])

In [109]:
len(rsp_json['explore_tabs'][0]['sections'])

4

In [113]:
rsp_json['explore_tabs'][0]['sections'][3].keys()

dict_keys(['backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'is_paginated', 'bankai_section_id', 'refinements', 'inserts', 'listings', 'review_items', 'breadcrumbs'])

In [115]:
len(rsp_json['explore_tabs'][0]['sections'][3]['listings'])

12

In [135]:
rsp_json['explore_tabs'][0]['sections'][3]['listings'][0];

In [205]:
flatten_json(rsp_json['explore_tabs'][0]['sections'][3]['listings'][0]);

## Functions

In [206]:
# fields

listing_fields = [
'bathrooms',
'bedrooms',
'beds',
'person_capacity',
# 'host_languages',
'id',
'is_business_travel_ready',
'is_fully_refundable',
'is_host_highly_rated',
'is_rebookable',
'is_superhost',
'lat',
'lng',
'picture_count',
'preview_amenities',
'reviews_count',
'star_rating',
'tier_id'
]

pricing_quote_rate_with_service_fee_fields = [
'amount',
'currency'
]

fields_default = ['listing_'+ field for field in listing_fields] + ['pricing_quote_rate_with_service_fee_'+field for field in pricing_quote_rate_with_service_fee_fields] + ['pricing_quote_rate_type']

In [207]:
fields_default

['listing_bathrooms',
 'listing_bedrooms',
 'listing_beds',
 'listing_person_capacity',
 'listing_id',
 'listing_is_business_travel_ready',
 'listing_is_fully_refundable',
 'listing_is_host_highly_rated',
 'listing_is_rebookable',
 'listing_is_superhost',
 'listing_lat',
 'listing_lng',
 'listing_picture_count',
 'listing_preview_amenities',
 'listing_reviews_count',
 'listing_star_rating',
 'listing_tier_id',
 'pricing_quote_rate_with_service_fee_amount',
 'pricing_quote_rate_with_service_fee_currency',
 'pricing_quote_rate_type']

In [217]:
def listing_parser(listing, fields = None):
    if fields == None:
        fields = ['listing_bathrooms', 'listing_bedrooms', 'listing_beds', 'listing_person_capacity', 'listing_host_languages', 'listing_id', 'listing_is_business_travel_ready', 'listing_is_fully_refundable', 'listing_is_host_highly_rated', 'listing_is_rebookable', 'listing_is_superhost', 'listing_lat', 'listing_lng', 'listing_picture_count', 'listing_preview_amenities', 'listing_reviews_count', 'listing_star_rating', 'listing_tier_id', 'pricing_quote_rate_with_service_fee_amount', 'pricing_quote_rate_with_service_fee_currency', 'pricing_quote_rate_type']
    
    return pd.DataFrame(listing)[fields_default]

        
    
    

In [221]:
def next_indicator(response_json):
    return response_json['explore_tabs'][0]['pagination_metadata']['has_next_page']

In [209]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

## Demo Scraping

In [244]:
next_sec = True
sec_off = 0
rsp = requests.get(api_base+api_key+'&section_offset=' + str(sec_off))
rsp_json = rsp.json()

listing_array = [flatten_json(item) for item in rsp_json['explore_tabs'][0]['sections'][3]['listings'] ]
df_total = listing_parser(listing_array)

df_total = pd.concat([df, df_total], join='inner')

next_sec = next_indicator(rsp_json)

sec_off = sec_off + 1

In [254]:
while next_sec == True:
    
    try_flag = 0

    while True and (try_flag < retry_lvl[0]):
        try:
            with requests.Session() as s:
                rsp = s.get(api_base+api_key+'&section_offset='+str(sec_off))
        except requests.exceptions.RequestException as err:
            print('request error', err)
            try_flag = try_flag + 1
            sleep(try_flag * retry_lvl[1])
            pass
        else:
            break

    rsp_json = rsp.json()
    

    listing_array = [flatten_json(item) for item in rsp_json['explore_tabs'][0]['sections'][0]['listings'] ]

    df = listing_parser(listing_array)
    df_total = pd.concat([df, df_total], join='inner')
    
    sec_off = sec_off + 1
    
    next_sec = next_indicator(rsp_json)
    

KeyError: "['listing_lat' 'listing_lng'] not in index"

In [255]:
df_total

,listing_bathrooms,listing_bedrooms,listing_beds,listing_person_capacity,listing_id,listing_is_business_travel_ready,listing_is_fully_refundable,listing_is_host_highly_rated,listing_is_rebookable,listing_is_superhost,listing_lat,listing_lng,listing_picture_count,listing_preview_amenities,listing_reviews_count,listing_star_rating,listing_tier_id,pricing_quote_rate_with_service_fee_amount,pricing_quote_rate_with_service_fee_currency,pricing_quote_rate_type
0,1.0,1,1,2,2787889,False,True,False,False,True,50.941302,6.936276,24,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",165,4.5,0,76.0,USD,nightly
1,1.0,1,1,1,24889964,False,True,False,False,False,50.932664,6.959956,9,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",1,NaN,0,37.0,USD,nightly
2,1.0,0,1,2,23365913,False,True,True,False,True,50.921521,6.933439,12,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",36,5.0,0,44.0,USD,nightly
3,1.0,1,2,3,277516,False,True,True,False,False,50.948113,6.950925,19,Kitchen,69,4.5,0,70.0,USD,nightly
4,1.0,2,3,6,151877,False,True,False,False,True,50.918680,6.962310,34,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",197,4.5,0,96.0,USD,nightly
5,1.0,1,2,2,10851686,False,True,True,False,True,50.918086,6.961938,6,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",65,5.0,0,37.0,USD,nightly
6,1.0,1,2,3,18425009,False,True,False,False,False,50.951706,6.956479,60,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",57,4.5,0,58.0,USD,nightly
7,1.0,1,2,3,21246774,False,True,True,False,True,50.938014,6.941539,14,"Wifi, Hair dryer",58,5.0,0,82.0,USD,nightly
8,1.0,1,1,2,10612377,False,True,False,False,False,50.941124,6.999401,15,"Wifi, Kitchen, Hair dryer, Laptop friendly wor...",74,4.5,0,26.0,USD,nightly
9,1.0,0,2,2,23447819,False,True,True,False,False,50.934467,6.973998,5,,36,5.0,0,35.0,USD,nightly


In [257]:
df_total.to_csv('data/cologne.csv')